In [7]:
import pandas as pd
import numpy as np


# Original dataset
csv_dir = "./data/monthly_crsp.csv"
# A relatively small dataset for testing
test_dir = "./data/test.csv"

df = pd.read_csv(csv_dir)

## Keep the following columns
- PERMNO(permno) - The unique identifier of the stock
- Ticker(ticker) - Abbreviation for stocks on the exchange
- NAICS(industry) - North American Industry Classification System
- MthCalDt(date) - Last trading day of the month
- MthRet(ret) - The stock return for the month
- sprtrn(ret_market) - S&P 500 monthly return

Permno is used to uniquely identify a stock. The ticker for the same permno may change. For example, permno=10001 had tickers [EGAS, EWST, GFGC], which are the same stock.


In [9]:
df = df[['PERMNO', 'Ticker', 'NAICS', 'MthCalDt', 'MthRet', 'sprtrn']]
df.columns = ['permno', 'ticker', 'industry', 'date', 'ret', 'ret_market']

If stock returns or market returns do not exist, then drop

In [11]:
df_clean = df.dropna(subset=['ret', 'ret_market'], how='any').copy()

- Change the time column to date format
- Keep data after 2000
- Drop duplicated rows

In [13]:
df_clean['date'] = pd.to_datetime(df_clean['date'])
df_clean = df_clean[df_clean['date'] >= '2000-01-01'].copy()
df_clean = df_clean.drop_duplicates(
    subset=['permno', 'date'], keep='first').copy()

A function for compute the forward returns.

In [15]:
def cum_forward(arr, n):
    values = np.asarray(arr)
    length = len(values)
    result = np.full(length, np.nan, dtype=float)
    for i in range(length - n + 1):
        result[i] = np.prod(values[i: i + n]) - 1
    return pd.Series(result, index=arr.index)

| Columns name   | Calculation formula | Interpretation     |
| ------ | ---- | -------- |
| ret_now_sign   |  1  if $r_t>0$ <br>  -1  if $r_t<0$|  The sign of the current month return    |
| ret_lag1   |  $r_{t-1}$  | The return of the  last month   |
|ret_trend1_sign|1 if $r_t>r_{t-1}$<br>-1 if $r_t<r_{t-1}$|Return increment sign|
|ret_cum_forward3m|$(r_t+1)(r_{t+1}+1)(r_{t+2}+1)-1$|Cumulative return for the next three months (including this month)|
|ret_cum_forward3m_sign|1 if ret_cum_forward3m>0<br>-1 if ret_cum_forward3m<0|The sign of ret_cum_forward3m|
|ret_cum_forward12m|$\sum_{\tau=0}^{11}(r_{t+\tau}+1)-1$|Cumulative return for the next 12 months (including this month)|
|ret_cum_forward12m_sign|1 if ret_cum_forward12m>0<br>-1 if ret_cum_forward12m<0|The sign of ret_cum_forward12m|
|ret_cum_backward3m|$(r_{t-1}+1)(r_{t-2}+1)(r_{t-3}+1)-1$|Cumulative return for the previous three months (not including this month)|
|ret_cum_backward6m|$\sum_{\tau=1}^{6}(r_{t-\tau}+1)-1$|Cumulative return for the previous six months (not including this month)
|ret_cum_backward12m|$\sum_{\tau=1}^{12}(r_{t-\tau}+1)-1$|Cumulative return for the previous 12 months (not including this month)

In [17]:
df = df_clean

df['ret_now_sign'] = df.apply(
    lambda row: (
        1 if (row['ret'] > 0)
        else (-1 if row['ret'] < 0
              else np.nan)
    ),
    axis=1
)

df['ret_lag1'] = df.groupby('permno')['ret'].shift(1)
df['ret_trend1_sign'] = df.apply(
    lambda row: (
        1 if (pd.notna(row['ret_lag1']) and row['ret'] > row['ret_lag1'])
        else (-1 if (pd.notna(row['ret_lag1']) and row['ret'] < row['ret_lag1'])
              else np.nan)
    ),
    axis=1
)

df.loc[:, 'ret_cum_forward3m'] = df.groupby('permno').apply(
    lambda grp: cum_forward(grp['ret'] + 1, 3),
    include_groups=False
).reset_index(level=0, drop=True)
df['ret_cum_forward3m_sign'] = df.apply(
    lambda row: (
        1 if (row['ret_cum_forward3m'] > 0)
        else (-1 if row['ret_cum_forward3m'] < 0
              else np.nan)
    ),
    axis=1
)

df.loc[:, 'ret_cum_forward12m'] = df.groupby('permno').apply(
    lambda grp: cum_forward(grp['ret'] + 1, 12),
    include_groups=False
).reset_index(level=0, drop=True)
df['ret_cum_forward12m_sign'] = df.apply(
    lambda row: (
        1 if (row['ret_cum_forward12m'] > 0)
        else (-1 if row['ret_cum_forward12m'] < 0
              else np.nan)
    ),
    axis=1
)

def cum_backward(arr, n):
    values = np.asarray(arr)
    length = len(values)
    result = np.full(length, np.nan, dtype=float)
    for i in range(n, length):
        result[i] = np.prod(values[i-n: i]) - 1
    return pd.Series(result, index=arr.index)


df.loc[:, 'ret_cum_backward3m'] = df.groupby('permno').apply(
    lambda grp: cum_backward(grp['ret'] + 1, 3),
    include_groups=False
).reset_index(level=0, drop=True)
df.loc[:, 'ret_cum_backward6m'] = df.groupby('permno').apply(
    lambda grp: cum_backward(grp['ret'] + 1, 6),
    include_groups=False
).reset_index(level=0, drop=True)
df.loc[:, 'ret_cum_backward12m'] = df.groupby('permno').apply(
    lambda grp: cum_backward(grp['ret'] + 1, 12),
    include_groups=False
).reset_index(level=0, drop=True)

In [18]:
df.head()

,permno,ticker,industry,date,ret,ret_market,ret_now_sign,ret_lag1,ret_trend1_sign,ret_cum_forward3m,ret_cum_forward3m_sign,ret_cum_forward12m,ret_cum_forward12m_sign,ret_cum_backward3m,ret_cum_backward6m,ret_cum_backward12m
187,10001,EWST,0,2000-01-31,-0.044118,-0.050904,-1.0,NaN,NaN,-0.044250,-1.0,0.215636,1.0,NaN,NaN,NaN
188,10001,EWST,0,2000-02-29,0.015385,-0.020108,1.0,-0.044118,1.0,0.011579,1.0,0.288048,1.0,NaN,NaN,NaN
189,10001,EWST,0,2000-03-31,-0.015288,0.096720,-1.0,0.015385,-1.0,-0.026827,-1.0,0.252475,1.0,NaN,NaN,NaN
190,10001,EWST,0,2000-04-28,0.011719,-0.030796,1.0,-0.015288,1.0,0.015564,1.0,0.321258,1.0,-0.044250,NaN,NaN
191,10001,EWST,0,2000-05-31,-0.023166,-0.021915,-1.0,0.011719,-1.0,-0.011884,-1.0,0.273304,1.0,0.011579,NaN,NaN


In [ ]:
df.to_csv('./data/target_data.csv', index=False)